In [19]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob


def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)  # Delete this line
    return undist

# prepare object points
nx = 9#TODO: enter the number of inside corners in x
ny = 6#TODO: enter the number of inside corners in y

imgpoints = []
objpoints = []

objp = np.zeros((ny*nx,3),np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2) #here we created a nice chessboard
filenames=glob.glob("camera_cal/calibration*.jpg")
    

for fname in filenames:
    img = cv2.imread(fname)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)# Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

    undistorted = cal_undistort(img, objpoints, imgpoints)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undistorted)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [21]:
%matplotlib qt
import cv2
import matplotlib.pyplot as plt
import numpy as np
img = cv2.imread("test_images/straight_lines1.jpg")
undistorted_straight = cal_undistort(img, objpoints, imgpoints)

plt.subplot(121)
plt.imshow(undistorted_straight)

plt.subplot(122)
plt.imshow(img)

plt.show()

In [23]:
#src = np.float32([[380,599], [915,599], [613, 440], [668,440]])
src = np.float32([[373,605], [920,605], [542, 487], [749,487]])
#dst = np.float32([[380,599], [915,599], [380, 440], [915,440]])
dst = np.float32([[373,605], [920,605], [373, 487], [920,487]])


img_size = (undistorted_straight.shape[1],undistorted_straight.shape[0])

M = cv2.getPerspectiveTransform(src,dst)
warped = cv2.warpPerspective(undistorted_straight,M,img_size,flags=cv2.INTER_LINEAR)

plt.imshow(warped)
plt.show()



In [29]:

exs2_img = cv2.imread("test_images/test4.jpg")
undistorted2 = cal_undistort(exs2_img, objpoints, imgpoints)
warped = cv2.warpPerspective(undistorted2,M,img_size,flags=cv2.INTER_LINEAR)

plt.subplot(121)
plt.imshow(warped)

plt.subplot(122)
plt.imshow(exs2_img)

plt.show()

In [41]:
def abs_sobel_thresh(img, orient='x', thresh_min=15, thresh_max=200):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

sobeled = abs_sobel_thresh(warped)
plt.imshow(sobeled, cmap="gray")
plt.show()
